## 1. install required packages

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180929144843-0001
Solving environment: done

# All requested packages already installed.



In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [3]:
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## 2. Get coordinates for Moscow neighborhoods by Nominatim service

Import file with names of Moscow neighborhoods

In [4]:
msk_csv = pd.read_csv('https://raw.githubusercontent.com/fedormalyshev/Moscow_Neighborhoods/master/Moscow_Neighborhoods.csv', encoding = "windows-1251")

In [5]:
msk_csv.head()

,Borough,Neighborhood
0,Центральный административный округ,район Арбат Москва
1,Центральный административный округ,Басманный район Москва
2,Центральный административный округ,район Замоскворечье Москва
3,Центральный административный округ,Красносельский район Москва
4,Центральный административный округ,Мещанский район Москва


In [6]:
df = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Lat', 'Lon'])

In [7]:
moscow_data = pd.concat([df, msk_csv])

In [8]:
moscow_data = moscow_data[['Borough', 'Neighborhood', 'Lat', 'Lon']]

In [9]:
moscow_data.head()

,Borough,Neighborhood,Lat,Lon
0,Центральный административный округ,район Арбат Москва,NaN,NaN
1,Центральный административный округ,Басманный район Москва,NaN,NaN
2,Центральный административный округ,район Замоскворечье Москва,NaN,NaN
3,Центральный административный округ,Красносельский район Москва,NaN,NaN
4,Центральный административный округ,Мещанский район Москва,NaN,NaN


In [10]:
for index, row in moscow_data.iterrows():
    geolocator = Nominatim()
    location = None
    while(location is None):
        try:
            location = geolocator.geocode(row['Neighborhood'])
        except:
            pass
    
    row['Lat'] = location.latitude
    row['Lon'] = location.longitude

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [11]:
moscow_data.head()

,Borough,Neighborhood,Lat,Lon
0,Центральный административный округ,район Арбат Москва,55.7512,37.5899
1,Центральный административный округ,Басманный район Москва,55.7742,37.6599
2,Центральный административный округ,район Замоскворечье Москва,55.7357,37.6349
3,Центральный административный округ,Красносельский район Москва,55.7784,37.6568
4,Центральный административный округ,Мещанский район Москва,55.7792,37.6278


In [12]:
moscow_data.to_csv('Moscow_Neighborhoods_LL.csv')

## 3. Map of Moscow with markers of neighborhoods 

In [13]:
geolocator = Nominatim()
location = geolocator.geocode('Москва')
moscow_latitude = location.latitude
moscow_longitude = location.longitude
print('The geograpical coordinate of Москва are {}, {}.'.format(moscow_latitude, moscow_longitude))

The geograpical coordinate of Москва are 55.7507178, 37.6176606.


/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [14]:
map_moscow = folium.Map(location=[moscow_latitude, moscow_longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(moscow_data['Lat'], moscow_data['Lon'], moscow_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)  
    
map_moscow

## 4. Getting venues info from Foursquare

In [25]:
# The code was removed by Watson Studio for sharing.

Venues in the first neighborhood

In [26]:
neighborhood_latitude = moscow_data.loc[0, 'Lat'] # neighborhood latitude value
neighborhood_longitude = moscow_data.loc[0, 'Lon'] # neighborhood longitude value
neighborhood_name = moscow_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of район Арбат Москва are 55.751199, 37.5898715.


In [27]:
LIMIT = 100
radius = 1000

In [28]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [29]:
results = requests.get(url).json()

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Театр им. Вахтангова,Theater,55.749650,37.591535
1,Corner Café & Kitchen,Japanese Restaurant,55.751496,37.586757
2,Obedbufet (Обедбуфет),Buffet,55.752268,37.592275
3,Baskin Robbins (Баскин Роббинс),Ice Cream Shop,55.753003,37.587662
4,Arbat Street (Арбат),Road,55.749941,37.593327


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Get venues for all neighborhoods

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
moscow_venues = getNearbyVenues(moscow_data.Neighborhood, moscow_data.Lat, moscow_data.Lon)

район Арбат Москва
Басманный район Москва
район Замоскворечье Москва
Красносельский район Москва
Мещанский район Москва
Пресненский район Москва
Таганский район Москва
Тверской район Москва
район Хамовники Москва
район Якиманка Москва
район Аэропорт Москва
район Беговой Москва
Бескудниковский район Москва
Войковский район Москва
район Восточное Дегунино Москва
Головинский район Москва
Дмитровский район Москва
район Западное Дегунино Москва
район Коптево Москва
район Левобережный Москва
Молжаниновский район Москва
Савёловский район Москва
район Сокол Москва
Тимирязевский район Москва
район Ховрино Москва
Хорошёвский район Москва
Алексеевский район Москва
Алтуфьевский район Москва
Бабушкинский район Москва
район Бибирево Москва
Бутырский район Москва
район Лианозово Москва
Лосиноостровский район Москва
район Марфино Москва
район Марьина Роща Москва
Останкинский район Москва
район Отрадное Москва
район Ростокино Москва
район Свиблово Москва
район Северный Москва
район Северное Медведково 

In [35]:
print(moscow_venues.shape)
moscow_venues[moscow_venues.Neighborhood=='район Арбат Москва']

(2366, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,район Арбат Москва,55.751199,37.589872,Театр им. Вахтангова,55.749650,37.591535,Theater
1,район Арбат Москва,55.751199,37.589872,Corner Café & Kitchen,55.751496,37.586757,Japanese Restaurant
2,район Арбат Москва,55.751199,37.589872,Obedbufet (Обедбуфет),55.752268,37.592275,Buffet
3,район Арбат Москва,55.751199,37.589872,Buffalo's - Wings and Nuggets,55.751840,37.587376,Wings Joint
4,район Арбат Москва,55.751199,37.589872,Кофемания,55.752094,37.588102,Coffee Shop
5,район Арбат Москва,55.751199,37.589872,Baskin Robbins (Баскин Роббинс),55.753003,37.587662,Ice Cream Shop
6,район Арбат Москва,55.751199,37.589872,Спортивный комплекс «Арбат»,55.750928,37.587130,Pool
7,район Арбат Москва,55.751199,37.589872,Граци Рагацци,55.752137,37.591267,Italian Restaurant
8,район Арбат Москва,55.751199,37.589872,Arbat Street (Арбат),55.749941,37.593327,Road
9,район Арбат Москва,55.751199,37.589872,Мемориальный музей А. Н. Скрябина,55.750792,37.590254,Museum


Let's check how many venues in each neighborhood:

In [36]:
moscow_venues.groupby('Neighborhood').size()

Neighborhood
Академический район Москва            31
Алексеевский район Москва             20
Алтуфьевский район Москва             10
Бабушкинский район Москва             39
Басманный район Москва                20
Бескудниковский район Москва          12
Бутырский район Москва                18
Войковский район Москва               44
Гагаринский район Москва              27
Головинский район Москва              11
Даниловский район Москва              36
Дмитровский район Москва              15
Донской район Москва                  18
Красносельский район Москва           14
Ломоносовский район Москва            45
Лосиноостровский район Москва         10
Мещанский район Москва                76
Можайский район Москва                 4
Молжаниновский район Москва            5
Нагорный район Москва                  9
Нижегородский район Москва            26
Обручевский район Москва              19
Останкинский район Москва             21
Пресненский район Москва              82
Ряз

We will consider only neighborhoods with more than 20 venues:

In [37]:
moscow_hood_cat = moscow_venues.groupby('Neighborhood').count()[['Venue Category']].reset_index()

In [54]:
moscow_hood_cat[moscow_hood_cat['Venue Category'] > 20]

,Neighborhood,Venue Category
0,Академический район Москва,31
3,Бабушкинский район Москва,39
7,Войковский район Москва,44
8,Гагаринский район Москва,27
10,Даниловский район Москва,36
14,Ломоносовский район Москва,45
16,Мещанский район Москва,76
20,Нижегородский район Москва,26
22,Останкинский район Москва,21
23,Пресненский район Москва,82


In [55]:
hoods2include = moscow_hood_cat[moscow_hood_cat['Venue Category'] > 20]['Neighborhood']
hoods2include

0              Академический район Москва
3               Бабушкинский район Москва
7                 Войковский район Москва
8                Гагаринский район Москва
10               Даниловский район Москва
14             Ломоносовский район Москва
16                 Мещанский район Москва
20             Нижегородский район Москва
22              Останкинский район Москва
23               Пресненский район Москва
26                 Таганский район Москва
27                  Тверской район Москва
29               Хорошёвский район Москва
30              Южнопортовый район Москва
32                     район Арбат Москва
34                   район Беговой Москва
48             район Замоскворечье Москва
53                 район Измайлово Москва
55                  район Коньково Москва
67              район Марьина Роща Москва
68                   район Марьино Москва
70                    район Митино Москва
80                  район Отрадное Москва
84      район Покровское-Стрешнево

In [56]:
moscow_venues2include = moscow_venues[moscow_venues['Neighborhood'].isin(hoods2include)]

In [57]:
moscow_venues2include.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,район Арбат Москва,55.751199,37.589872,Театр им. Вахтангова,55.749650,37.591535,Theater
1,район Арбат Москва,55.751199,37.589872,Corner Café & Kitchen,55.751496,37.586757,Japanese Restaurant
2,район Арбат Москва,55.751199,37.589872,Obedbufet (Обедбуфет),55.752268,37.592275,Buffet
3,район Арбат Москва,55.751199,37.589872,Buffalo's - Wings and Nuggets,55.751840,37.587376,Wings Joint
4,район Арбат Москва,55.751199,37.589872,Кофемания,55.752094,37.588102,Coffee Shop


In [58]:
print('There are {} uniques categories.'.format(len(moscow_venues2include['Venue Category'].unique())))

There are 228 uniques categories.


## 5. Analyze Each Neighborhood

In [59]:
# one hot encoding
moscow_onehot = pd.get_dummies(moscow_venues2include[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
moscow_onehot['Neighborhood'] = moscow_venues2include['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]

moscow_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,район Арбат Москва,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,район Арбат Москва,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,район Арбат Москва,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,район Арбат Москва,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,район Арбат Москва,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
moscow_onehot.shape

(1472, 229)

## 6. Group rows by neighborhood and by taking the mean of the frequency of occurence of each category

In [61]:
moscow_grouped = moscow_onehot.groupby('Neighborhood').mean().reset_index()
moscow_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Академический район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.000000
1,Бабушкинский район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Войковский район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022727,0.000000,0.000000
3,Гагаринский район Москва,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.037037,0.00,0.000000,0.000000,0.000000
4,Даниловский район Москва,0.000000,0.000000,0.000000,0.000000,0.138889,0.000000,0.000000,0.000000,0.027778,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Ломоносовский район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Мещанский район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,...,0.00,0.013158,0.000000,0.013158,0.000000,0.000000,0.00,0.000000,0.013158,0.000000
7,Нижегородский район Москва,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.038462,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Останкинский район Москва,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.047619,0.00,0.000000,0.000000,0.000000
9,Пресненский район Москва,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.00,0.012195,0.000000,0.012195,0.024390,0.000000,0.00,0.000000,0.012195,0.000000


In [62]:
moscow_grouped.shape

(36, 229)

## 7. Print each neighborhood along with the top 5 most common venues

In [63]:
num_top_venues = 5

for hood in moscow_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = moscow_grouped[moscow_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Академический район Москва----
           venue  freq
0       Pharmacy  0.10
1  Grocery Store  0.06
2           Park  0.06
3   Liquor Store  0.03
4       Bus Line  0.03


----Бабушкинский район Москва----
                  venue  freq
0              Bus Stop  0.08
1                  Café  0.05
2      Sushi Restaurant  0.05
3        Cosmetics Shop  0.05
4  Fast Food Restaurant  0.05


----Войковский район Москва----
                 venue  freq
0       Clothing Store  0.14
1       Lingerie Store  0.07
2  Sporting Goods Shop  0.07
3           Shoe Store  0.07
4     Toy / Game Store  0.05


----Гагаринский район Москва----
                venue  freq
0            Pharmacy  0.07
1  Italian Restaurant  0.07
2    Toy / Game Store  0.04
3                Café  0.04
4                Park  0.04


----Даниловский район Москва----
                       venue  freq
0                     Arcade  0.14
1                Coffee Shop  0.06
2       Fast Food Restaurant  0.06
3  Middle Eastern Restaur

## 8. Create new dataframe and display the top 10 venue categories for each neighborhood 

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = moscow_grouped['Neighborhood']

for ind in np.arange(moscow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Академический район Москва,Pharmacy,Park,Grocery Store,Sporting Goods Shop,Sandwich Place,Music Store,Soccer Field,Fast Food Restaurant,Bus Line,Bus Stop
1,Бабушкинский район Москва,Bus Stop,Fast Food Restaurant,Café,Coffee Shop,Pharmacy,Cosmetics Shop,Caucasian Restaurant,Sushi Restaurant,Plaza,Paper / Office Supplies Store
2,Войковский район Москва,Clothing Store,Sporting Goods Shop,Lingerie Store,Shoe Store,Restaurant,Toy / Game Store,Convenience Store,Food & Drink Shop,Miscellaneous Shop,Furniture / Home Store
3,Гагаринский район Москва,Italian Restaurant,Pharmacy,Sports Club,Toy / Game Store,Sporting Goods Shop,Furniture / Home Store,Café,Steakhouse,Electronics Store,Bike Rental / Bike Share
4,Даниловский район Москва,Arcade,Middle Eastern Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Steakhouse,Auto Workshop,Sporting Goods Shop,Big Box Store,Blini House
5,Ломоносовский район Москва,Pharmacy,Kids Store,Convenience Store,Flower Shop,Health Food Store,Pedestrian Plaza,Bakery,Gym / Fitness Center,Café,Bookstore
6,Мещанский район Москва,Coffee Shop,Restaurant,Middle Eastern Restaurant,Dance Studio,Bakery,Spa,Caucasian Restaurant,Cosmetics Shop,Pizza Place,Museum
7,Нижегородский район Москва,Clothing Store,Arcade,Fast Food Restaurant,Furniture / Home Store,Bakery,Sporting Goods Shop,Bus Stop,Shopping Mall,Boutique,Blini House
8,Останкинский район Москва,Scenic Lookout,Design Studio,Pharmacy,Café,Park,Garden Center,Supermarket,Bus Stop,Gym / Fitness Center,Tram Station
9,Пресненский район Москва,Coffee Shop,Café,Gym / Fitness Center,Board Shop,Caucasian Restaurant,Music Venue,Dance Studio,Bar,Wine Bar,Camera Store


## 9. Cluster neighborhoods

In [70]:
# set number of clusters
kclusters = 3

moscow_grouped_clustering = moscow_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 1, 1, 0, 1, 0, 2, 1, 0, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 2, 2, 1, 1, 2, 1], dtype=int32)

In [71]:
moscow_data2include = moscow_data[moscow_data['Neighborhood'].isin(hoods2include)]

In [72]:
moscow_merged = neighborhoods_venues_sorted

# add clustering labels
moscow_merged['Cluster Labels'] = kmeans.labels_

moscow_merged = moscow_merged.join(moscow_data2include.set_index('Neighborhood'), on='Neighborhood')

moscow_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Lat,Lon
0,Академический район Москва,Pharmacy,Park,Grocery Store,Sporting Goods Shop,Sandwich Place,Music Store,Soccer Field,Fast Food Restaurant,Bus Line,Bus Stop,0,Юго-Западный административный округ,55.6894,37.578
1,Бабушкинский район Москва,Bus Stop,Fast Food Restaurant,Café,Coffee Shop,Pharmacy,Cosmetics Shop,Caucasian Restaurant,Sushi Restaurant,Plaza,Paper / Office Supplies Store,1,Северо-Восточный административный округ,55.8672,37.6631
2,Войковский район Москва,Clothing Store,Sporting Goods Shop,Lingerie Store,Shoe Store,Restaurant,Toy / Game Store,Convenience Store,Food & Drink Shop,Miscellaneous Shop,Furniture / Home Store,1,Северный административный округ,55.8279,37.4957
3,Гагаринский район Москва,Italian Restaurant,Pharmacy,Sports Club,Toy / Game Store,Sporting Goods Shop,Furniture / Home Store,Café,Steakhouse,Electronics Store,Bike Rental / Bike Share,0,Юго-Западный административный округ,55.6981,37.5579
4,Даниловский район Москва,Arcade,Middle Eastern Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Steakhouse,Auto Workshop,Sporting Goods Shop,Big Box Store,Blini House,1,Южный административный округ,55.7068,37.6382


## 10. Map with the resulting clusters

In [73]:
# create map
map_clusters = folium.Map(location=[moscow_latitude, moscow_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys*2)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Lat'], moscow_merged['Lon'], moscow_merged['Neighborhood'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters